In [140]:
import pandas as pd
import numpy as np

data = pd.read_csv("./data/ufos.csv")
data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611


In [141]:
ufo_data = pd.DataFrame({ "duration": data["duration (seconds)"], "lat": data["latitude"], "lng": data["longitude"], "country": data["country"] })

ufo_data.country.unique()

array(['us', nan, 'gb', 'ca', 'au', 'de'], dtype=object)

In [142]:
ufo_data.dropna(inplace=True)
ufo_data2 = ufo_data[(ufo_data.duration <= 60) & (ufo_data.duration >= 1)]
ufo_data2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25863 entries, 2 to 80330
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   duration  25863 non-null  float64
 1   lat       25863 non-null  float64
 2   lng       25863 non-null  float64
 3   country   25863 non-null  object 
dtypes: float64(3), object(1)
memory usage: 1010.3+ KB


In [143]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

ufo_data2.country = label_encoder.fit_transform(ufo_data2.country)

ufo_data2.head()

/tmp/ipykernel_742/1231013484.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ufo_data2.country = label_encoder.fit_transform(ufo_data2.country)


,duration,lat,lng,country
2,20.0,53.200000,-2.916667,3
3,20.0,28.978333,-96.645833,4
14,30.0,35.823889,-80.253611,4
23,60.0,45.582778,-122.352222,4
24,3.0,51.783333,-0.783333,3


In [144]:
from sklearn.model_selection import train_test_split

x = ufo_data2[["duration", "lat", "lng"]]
y = ufo_data2.country

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=False)

In [145]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

pred = model.predict(x_test)
print(classification_report(y_test, pred))
print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       0.85      0.47      0.60       250
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00       131
           4       0.97      1.00      0.98      4743

    accuracy                           0.97      5173
   macro avg       0.96      0.89      0.92      5173
weighted avg       0.97      0.97      0.97      5173

0.9702300405953992
[[  41    0    0    0    0]
 [   0  117    0    0  133]
 [   0    0    8    0    0]
 [   0    0    0  131    0]
 [   0   21    0    0 4722]]


/home/vlad/experiments/ML-For-Beginners/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [148]:
import pickle
pickle.dump(model, open("ufo-model.pkl", "wb"))

restored_model = pickle.load(open("ufo-model.pkl", "rb"))

label_encoder.inverse_transform(restored_model.predict([[50, 12, 12]]))

/home/vlad/experiments/ML-For-Beginners/.venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array(['de'], dtype=object)